# NHL API Data Extraction

Welcome to my NHL data extraction project! I wanted to challenge myself with a new data project, which would help me build skills in handling data. I wanted to focus on a Extract, Transform and Load process for data that was in an unfamiliar form to me. This happened to be perfect as I just spent the past few weeks with the NHL API. This notebook will pull data from the the NHL API, which will then be formatted to my preference, and loading to a database, where I will be creating a visualization with this data.

I would also like to thank anyone who has taken the time to look over this project. If you have any feedback or questions, feel free to message me.

In [1]:
#Imports
import json
import pandas as pd
import requests
from sqlalchemy import create_engine


In [2]:
# Function to acquire json file
# Referenced from https://colab.research.google.com/drive/1j37IaWSCM1EpqfM1h2kfvq2ncha_hqG8?usp=sharing#scrollTo=EEzfwD9bZxab

def get_json(url):
  try:
    response = requests.get(url, params={"Content-Type": "application/json"})
    response.raise_for_status()
  except requests.exceptions.HTTPError as errh:
    print ("Http Error:",errh)
    return
  except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
    return
  except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
    return
  except requests.exceptions.RequestException as err:
    print ("Other Error",err)
    return
  data = response.json()
  return data

# Function to print json data nicely
def print_json(data):
  print(json.dumps(data, indent=2))

In [3]:
# Create DF of Current Standings

url = 'https://api-web.nhle.com/v1/standings/now'
data = get_json(url)

standings_df = pd.DataFrame(data['standings'])
standings_df['date'] = pd.to_datetime(standings_df['date'])

if 'teamName' in standings_df.columns:
    standings_df['teamName'] = standings_df['teamName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
if 'teamCommonName' in standings_df.columns:
    standings_df['teamCommonName'] = standings_df['teamCommonName'].apply(lambda x: x['default'] if isinstance(x, dict) else x)
if 'teamAbbrev' in standings_df.columns:
    standings_df['teamAbbrev'] = standings_df['teamAbbrev'].apply(lambda x: x['default'] if isinstance(x, dict) else x)

# Team Abbrevs used to cycle through games
team_abbrevs = [standings_df['teamAbbrev'].unique()]


In [4]:
# Function to get season id based on Min Date Game
# Season ids in the format of yyyyyyyy

def set_season_id(standingsdf):
    min_date = min(standings_df['date'])
    '''
    Get the minimum date for standings
    
    The min date should be the day the notebook was run, or the end of the season.
    
    August is the turnover point, to switch to the next season.
    
    i.e seasonid = 20232024
    '''
    year = min_date.year
    if min_date.month < 8:
        season_id = f"{year-1}{year}"
    else:
        season_id = f"{year}{year+1}"
    return season_id

In [5]:
# Get Games from Most recent Season

season_id = set_season_id(standings_df)
game_ids = set() # Set so only contains unique IDs

for team in team_abbrevs[0]:
        url = f"https://api-web.nhle.com/v1/club-schedule-season/{team}/{season_id}"
        data = get_json(url)
        for game in data['games']:
            if game['gameState'] == 'OFF':
                if game['gameType'] in (2,3): #Only get regular season and playoffs (gametype 2 and 3 respectively)
                    id = game['id']
                    if id not in game_ids:
                        game_ids.add(id)

print(f'{len(game_ids)} Game Ids added to Game_ids')

1135 Game Ids added to Game_ids


In [6]:
# Get stat function will simplify getting game stats that are deeply nested in the JSON
def get_stat(teamGameStats, stat_category, is_home):
    stat_key = 'homeValue' if is_home else 'awayValue'
    for stat in teamGameStats:
        if stat['category'] == stat_category:
            return stat[stat_key]
    return None


def game_stat_extraction(gameid):
    url = f'https://api-web.nhle.com/v1/gamecenter/{gameid}/boxscore'
    boxscore = get_json(url)
    
    boxscore_df = []
    
    home = boxscore.get('homeTeam',None)
    away = boxscore.get('awayTeam', None)
    teamGameStats = boxscore['summary']['teamGameStats']
    
    game_details = {
    'gameid' : gameid,
    'venue': boxscore['venue']['default'],
    'gameType': boxscore['gameType'],
    'season': boxscore['season'],
    'period_count': boxscore['periodDescriptor']['number'],
    'period_type': boxscore['periodDescriptor']['periodType'],
    'home_id': home.get('id'),
    'home_goals': home.get('score'),
    'home_sog': get_stat(teamGameStats, 'sog', True),
    'home_faceoffWinningPctg': get_stat(teamGameStats, 'faceoffWinningPctg', True),
    'home_powerPlayConversion': get_stat(teamGameStats, 'powerPlay', True),
    'home_pim': get_stat(teamGameStats, 'pim', True),
    'home_hits': get_stat(teamGameStats, 'hits', True),
    'home_blocks': get_stat(teamGameStats, 'blockedShots', True),
    'away_id': away.get('id'),
    'away_goals': away.get('score', None),
    'away_sog': get_stat(teamGameStats, 'sog', False),
    'away_faceoffWinningPctg': get_stat(teamGameStats, 'faceoffWinningPctg', False),
    'away_powerPlayConversion': get_stat(teamGameStats, 'powerPlay', False),
    'away_pim': get_stat(teamGameStats, 'pim', False),
    'away_hits': get_stat(teamGameStats, 'hits', False),
    'away_blocks': get_stat(teamGameStats, 'blockedShots', False),
    }
    
    boxscore_df.append(game_details)
    boxscore_df = pd.DataFrame(boxscore_df)
    
    return boxscore_df


In [7]:
# For loop will iterate over all of our game ids, get their boxscore and add them to the season_boxscores dataframe

season_boxscores = pd.DataFrame()

for game_id in game_ids:
    try:
        temp_df = game_stat_extraction(game_id)
        season_boxscores = pd.concat([season_boxscores, temp_df], ignore_index=True)
    except requests.HTTPError as e:
        print(f"HTTP Error for game ID {game_id}: {e}")  # Log the error or print it
    except TypeError as e:
        print(f"TypeError encountered for game ID {game_id}: {e}")  # Handle cases where data might be None
    except Exception as e:
        print(f"Unexpected error for game ID {game_id}: {e}")
        
# season_boxscores now contains all team stats for their games played

In [8]:
# Get player info and game logs in seperate dfs

# Get goalie and skater ID' to later get their stats

player_ids = set()
goalie_ids = set()
season_id = set_season_id(standings_df)

for team in team_abbrevs[0]:
    url = f'https://api-web.nhle.com/v1/roster/{team}/20222023'
    try:
        data = get_json(url)

        for position_group in data.values():
            if isinstance(position_group, list):
                for player in position_group:
                    player_id = player.get('id')
                    position_code = player.get('positionCode')
                    if player_id:
                        if position_code == "G":
                            goalie_ids.add(player_id)
                        else:
                            player_ids.add(player_id)
    except requests.RequestException as e:
        print(f"Request error for team {team}: {e}")
        

# Player stats

# Initialize list, which will later be appended to the aggregated player stats df
# Player and Goalie stats will be gathered from the player landing endpoint

player_stats_list = []

for player_id in player_ids:
    url = f"https://api-web.nhle.com/v1/player/{player_id}/landing"
    data = get_json(url)
    
    season_stats = data['featuredStats']['regularSeason'].get('subSeason', {})
            
    season_stats_all = {
                'player_id': player_id,
                'current_team': data.get('currentTeamId',None),
                'first_name': data['firstName']['default'],
                'last_name': data['lastName']['default'],
                'position': data['position'],
                'photo': data['headshot'],
                'games_played': season_stats.get('gamesPlayed'),
                'goals': season_stats.get('goals'),
                'assists': season_stats.get('assists'),
                'points': season_stats.get('points'),
                'plus_minus': season_stats.get('plusMinus'),
                'pim': season_stats.get('pim'),
                'game_winning_goals': season_stats.get('gameWinningGoals'),
                'ot_goals': season_stats.get('otGoals'),
                'shots': season_stats.get('shots'),
                'shooting_pctg': season_stats.get('shootingPctg'),
                'power_play_goals': season_stats.get('powerPlayGoals'),
                'power_play_points': season_stats.get('powerPlayPoints'),
                'shorthanded_goals': season_stats.get('shorthandedGoals'),
                'shorthanded_points': season_stats.get('shorthandedPoints'),
            }
            
    player_stats_list.append(season_stats_all)
player_stats = pd.DataFrame(player_stats_list)


# Goalie Stats
goalie_stats_list = []

for goalie_id in goalie_ids:
    url = f"https://api-web.nhle.com/v1/player/{goalie_id}/landing"
    data = get_json(url)

    season_stats = data['featuredStats']['regularSeason'].get('subSeason', {})
            
    season_stats_all = {
        'goalie_id': goalie_id,
        'current_team': data.get('currentTeamId',None),
        'first_name': data['firstName']['default'],
        'last_name': data['lastName']['default'],
        'position': data['position'],
        'photo': data['headshot'],
        'games_played': season_stats.get('gamesPlayed'),
        'wins': season_stats.get('wins'),
        'losses': season_stats.get('losses'),
        'ties': season_stats.get('ties'),
        'otLosses': season_stats.get('otLosses'),
        'shutouts': season_stats.get('shutouts'),
        'goalsAgainstAvg': season_stats.get('goalsAgainstAvg'),
        'savePctg': season_stats.get('savePctg')
            }
            
    goalie_stats_list.append(season_stats_all)
            
goalie_stats = pd.DataFrame(goalie_stats_list)

In [9]:
# Correcting formatting issue with object dtypes and the column placeName in standings_df

for col in standings_df.select_dtypes(include=['object']).columns:
    standings_df[col] = standings_df[col].apply(lambda x: x.strip().replace(r'[^\x00-\x7F]+', '') if isinstance(x, str) else json.dumps(x))

standings_df['placeName'] = standings_df['placeName'].apply(lambda x: json.loads(x)['default'] if isinstance(x, str) else x)


In [10]:
# Create connection to Database.

db_config = {
    "username": "username",
    "password": "password",
    "host": "hose",
    "port": 'port',
    "database": "nhl"
}
connection_string = f"mysql+pymysql://{db_config['username']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)


In [13]:
'''
TABLES TO BE UPLOADED

Standings = standings_df
Boxscores = season_boxscores
Players = player_stats
Goalies = goalie_stats

'''
# Upload and append to tables

standings_df.to_sql('standings', con=engine, index=False, if_exists='append')
season_boxscores.to_sql('boxscores', con=engine, index=False, if_exists='append')
player_stats.to_sql('players', con=engine, index=False, if_exists='replace')
goalie_stats.to_sql('goalies', con=engine, index=False, if_exists='replace')

107

## Conclusion

This project has been interesting to work on, I was able to understand the process of how to gather data from a source such as an API, and process it for use. I learned the fundamentals of working with API endpoints, JSON files and connecting to a Database (A lot easier than I thought). If I was to improve this project, I would like to build out an advanced schema for the database. I would also like to create simple and efficient ways to continue uploading new data to the database.

As a loose schema, here are the talbes, and the respective 'teamID' shared across them.

Table      | Keys
-----------| -------------
Standings  | TeamId
Boxscores  | TeamId
Players    | TeamId
Goalies    | TeamId


This data extraction process is not flashy, but it will compliment my next project where I work with the data in a visualization software.

The tables of the 4 dataframes will be shown below, just for any curious to what the data looks like.

Thank you for checking out my project!

In [18]:
standings_df.head(5)

,conferenceAbbrev,conferenceHomeSequence,conferenceL10Sequence,conferenceName,conferenceRoadSequence,conferenceSequence,date,divisionAbbrev,divisionHomeSequence,divisionL10Sequence,...,streakCount,teamName,teamCommonName,teamAbbrev,teamLogo,ties,waiversSequence,wildcardSequence,winPctg,wins
0,E,3,3,Eastern,3,1,2024-03-26,M,2,2,...,2,New York Rangers,Rangers,NYR,https://assets.nhle.com/logos/nhl/svg/NYR_ligh...,0,32,0,0.661972,47
1,W,2,4,Western,3,1,2024-03-26,P,1,1,...,1,Vancouver Canucks,Canucks,VAN,https://assets.nhle.com/logos/nhl/svg/VAN_ligh...,0,29,0,0.625000,45
2,E,6,6,Eastern,1,2,2024-03-26,A,4,3,...,1,Florida Panthers,Panthers,FLA,https://assets.nhle.com/logos/nhl/svg/FLA_ligh...,0,31,0,0.647887,46
3,W,1,2,Western,6,2,2024-03-26,C,1,2,...,9,Colorado Avalanche,Avalanche,COL,https://assets.nhle.com/logos/nhl/svg/COL_ligh...,0,30,0,0.647887,46
4,E,1,1,Eastern,5,3,2024-03-26,M,1,1,...,1,Carolina Hurricanes,Hurricanes,CAR,https://assets.nhle.com/logos/nhl/svg/CAR_ligh...,0,28,0,0.625000,45


In [19]:
season_boxscores.head(5)

,gameid,venue,gameType,season,period_count,period_type,home_id,home_goals,home_sog,home_faceoffWinningPctg,...,home_hits,home_blocks,away_id,away_goals,away_sog,away_faceoffWinningPctg,away_powerPlayConversion,away_pim,away_hits,away_blocks
0,2023020544,T-Mobile Arena,2,20232024,3,REG,54,3,32,0.442623,...,24,34,26,2,34,0.557377,0/2,0,26,9
1,2023020545,SAP Center at San Jose,2,20232024,3,REG,28,0,25,0.560000,...,19,15,22,5,30,0.440000,0/1,6,11,12
2,2023020546,Nationwide Arena,2,20232024,4,OT,29,6,21,0.428571,...,14,12,10,5,33,0.571429,2/4,11,13,17
3,2023020547,Little Caesars Arena,2,20232024,4,OT,17,5,23,0.492063,...,25,18,18,4,37,0.507937,0/2,11,22,15
4,2023020548,Amerant Bank Arena,2,20232024,3,REG,13,4,33,0.576271,...,34,12,3,3,34,0.423729,0/2,6,24,23


In [20]:
player_stats.head(5)

,player_id,current_team,first_name,last_name,position,photo,games_played,goals,assists,points,plus_minus,pim,game_winning_goals,ot_goals,shots,shooting_pctg,power_play_goals,power_play_points,shorthanded_goals,shorthanded_points
0,8474628,NaN,Michael,Stone,D,https://assets.nhle.com/mugs/nhl/latest/847462...,48,6,5,11,2,35,1,0,92,0.065217,0,2,0,0
1,8480776,54.0,Sheldon,Rempal,R,https://assets.nhle.com/mugs/nhl/20232024/VGK/...,9,2,1,3,-2,4,0,0,16,0.125000,1,1,0,0
2,8482824,23.0,Nick,Cicek,D,https://assets.nhle.com/mugs/nhl/20232024/VAN/...,16,0,4,4,-7,15,0,0,3,0.000000,0,0,0,0
3,8474641,22.0,Adam,Henrique,C,https://assets.nhle.com/mugs/nhl/20232024/EDM/...,69,20,25,45,5,35,3,0,120,0.166667,5,10,2,5
4,8482834,NaN,Jesper,Froden,R,https://assets.nhle.com/mugs/nhl/latest/848283...,14,0,4,4,4,6,0,0,10,0.000000,0,0,0,0


In [21]:
goalie_stats.head(5)

,goalie_id,current_team,first_name,last_name,position,photo,games_played,wins,losses,ties,otLosses,shutouts,goalsAgainstAvg,savePctg
0,8482821,16.0,Arvid,Soderblom,G,https://assets.nhle.com/mugs/nhl/20232024/CHI/...,27,4,20,0.0,1,0,4.016979,0.876289
1,8480263,NaN,Dylan,Ferguson,G,https://assets.nhle.com/mugs/nhl/latest/848026...,2,1,1,NaN,0,0,2.523836,0.939759
2,8475660,26.0,Cam,Talbot,G,https://assets.nhle.com/mugs/nhl/20232024/LAK/...,46,23,16,0.0,6,3,2.424990,0.917381
3,8474636,17.0,Michael,Hutchinson,G,https://assets.nhle.com/mugs/nhl/20232024/DET/...,1,0,1,0.0,0,0,3.114187,0.916667
4,8480280,6.0,Jeremy,Swayman,G,https://assets.nhle.com/mugs/nhl/20232024/BOS/...,39,22,8,0.0,8,3,2.571391,0.916881
